In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
# Variables
random_state = 42
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
DetectorFactory.seed = random_state

# Sklearn
method = 'BERT'
final_models_save_path = f'{models_save_path}{method} Results/'
t = time.time()
n_jobs = -1
n_splits = 10
n_repeats = 3
class_weight = 'balanced'
cv = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
t = time.time()
cores = multiprocessing.cpu_count()
scoring = 'recall'
scores = ['recall', 'accuracy', 'f1', 'roc_auc',
          'explained_variance', 'matthews_corrcoef']
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score),
}
analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'
metrics_dict = {
    'Mean Cross Validation Train Score': np.nan,
    'Mean Cross Validation Test Score': np.nan,
    f'Mean Explained Train Variance - {scoring.title()}': np.nan,
    f'Mean Explained Test Variance - {scoring.title()}': np.nan,
    'Explained Variance': np.nan,
    'Accuracy': np.nan,
    'Balanced Accuracy': np.nan,
    'Precision': np.nan,
    'Recall': np.nan,
    'F1-score': np.nan,
    'Matthews Correlation Coefficient': np.nan,
    'Fowlkes–Mallows Index': np.nan,
    'ROC': np.nan,
    'AUC': np.nan,
    f'{scoring.title()} Best Threshold': np.nan,
    f'{scoring.title()} Best Score': np.nan,
    'Log Loss/Cross Entropy': np.nan,
    'Cohen’s Kappa': np.nan,
    'Geometric Mean': np.nan,
    'Classification Report': np.nan,
    'Confusion Matrix': np.nan,
    'Normalized Confusion Matrix': np.nan
}

# Plotting
plt.style.use('tableau-colorblind10')
plt.set_cmap('PuBu_r')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 5000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.2f}'.format)

# BERT variables
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizerFast.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertForSequenceClassification.from_pretrained(bert_model_name).to(device)



# Functions

In [ ]:
def show_and_close_plots():
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()


In [ ]:
def close_plots():
    plt.clf()
    plt.cla()
    plt.close()


In [ ]:
def split_data(df, col, analysis_columns, text_col=None):

    if text_col is None:
        text_col = 'Job Description spacy_sentencized'

    train_ratio = 0.75
    test_ratio = 0.10
    validation_ratio = 0.15
    test_split = test_size = 1 - train_ratio
    validation_split = test_ratio / (test_ratio + validation_ratio)

    # Split
    print('='*20)
    print('Splitting data into training, testing, and validation sets:')
    print(f'Ratios: train_size = {train_ratio}, test size = {test_ratio}, validation size = {validation_ratio}')

    df = df.dropna(subset=analysis_columns, how='any')
    df = df.reset_index(drop=True)

    train, test = train_test_split(
        df, train_size = 1-test_split, test_size = test_split, random_state=random_state
    )

    val, test = train_test_split(
        test, test_size=validation_split, random_state=random_state
    )

    X_train = np.array(list(train[text_col].astype('str').values))
    y_train = column_or_1d(train[col].astype('int64').values.tolist(), warn=True)

    X_test = np.array(list(test[text_col].astype('str').values))
    y_test = column_or_1d(test[col].astype('int64').values.tolist(), warn=True)

    X_val = np.array(list(val[text_col].astype('str').values))
    y_val = column_or_1d(val[col].astype('int64').values.tolist(), warn=True)

    class_weights = compute_class_weight(class_weight = 'balanced', classes = [0,1], y = y_train)
    class_weights_ratio = class_weights[0]/class_weights[1]
    class_weights_dict = dict(zip(np.unique(y_train), class_weights))

    print('Done splitting data into training, testing, and validation sets.')

    return (
        train, X_train, y_train,
        test, X_test, y_test,
        val, X_val, y_val,
        class_weights,
        class_weights_ratio,
        class_weights_dict
    )


In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, encoded):
        self.encodings = encodings
        self.encoded = encoded

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.encoded[idx], device=device)
        return item

    def __len__(self):
        return len(self.encoded)


In [ ]:
def encode_data(df, col, analysis_columns, text_col=None):

    # Split
    (
        train, X_train, y_train,
        test, X_test, y_test,
        val, X_val, y_val,
        class_weights,
        class_weights_ratio,
        class_weights_dict
    ) = split_data(
        df=df_manual, col=col, analysis_columns=analysis_columns, text_col=text_col
    )

    if text_col is None:
        text_col = 'Job Description spacy_sentencized'

    print('='*20)
    print(f'Encoding training, testing, and validation sets with {bert_tokenizer.__class__.__name__}.from_pretrained using {bert_tokenizer.name_or_path}.')

    bert_label2id = {label: id_ for id_, label in enumerate(set(label for label in y_train))}
    bert_id2label = {id_: label for label, id_ in bert_label2id.items()}

    X_train_bert_encodings = bert_tokenizer(
        X_train.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
    ).to(device)
    y_train_bert_encoded = [bert_label2id[y] for y in y_train]
    bert_train_dataset = MyDataset(X_train_bert_encodings, y_train_bert_encoded)

    X_test_bert_encodings = bert_tokenizer(
        X_test.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
    ).to(device)
    y_test_bert_encoded = [bert_label2id[y] for y in y_test]
    bert_test_dataset = MyDataset(X_test_bert_encodings, y_test_bert_encoded)

    X_val_bert_encodings = bert_tokenizer(
        X_val.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
    ).to(device)
    y_val_bert_encoded = [bert_label2id[y] for y in y_val]
    bert_val_dataset = MyDataset(X_val_bert_encodings, y_val_bert_encoded)

    print('Done encoding training, testing, and validation sets.')
    print('='*20)
    print(f'Training set shape: {y_train.shape}')
    print('-'*10)
    print(f'Training set example:\n{X_train[0]}')
    print('-'*10)
    print(f'Training set BERT encodings example:\n{" ".join(bert_train_dataset.encodings[0].tokens[:30])}')
    print('-'*10)
    print(f'Training labels after BERT encoding: {set(y_train_bert_encoded)}')
    print('~'*10)
    print(f'Testing set shape: {y_test.shape}')
    print('-'*10)
    print(f'Testing set example:\n{X_test[0]}')
    print('-'*10)
    print(f'Testing set BERT encodings example:\n{" ".join(bert_test_dataset.encodings[0].tokens[:30])}')
    print('-'*10)
    print(f'Testing labels after BERT encoding: {set(y_test_bert_encoded)}')
    print('~'*10)
    print(f'Validation set shape: {y_val.shape}')
    print('-'*10)
    print(f'Validation set example:\n{X_val[0]}')
    print('-'*10)
    print(f'Validation set BERT encodings example:\n{" ".join(bert_val_dataset.encodings[0].tokens[:30])}')
    print('-'*10)
    print(f'Validation labels after BERT encoding: {set(y_val_bert_encoded)}')
    print('~'*10)
    print(f'Class weights:\nRatio = {class_weights_ratio:.2f} (0 = {class_weights[0]:.2f}, 1 = {class_weights[1]:.2f})')
    print('='*20)

    return (
        train, X_train, X_train_bert_encodings, y_train, y_train_bert_encoded, bert_train_dataset,
        test, X_test, X_test_bert_encodings, y_test, y_test_bert_encoded, bert_test_dataset,
        val, X_val, X_val_bert_encodings, y_val, y_val_bert_encoded, bert_val_dataset,
        bert_label2id, bert_id2label, class_weights, class_weights_ratio, class_weights_dict
    )


In [ ]:
def compute_metrics_with_y_pred(
    y_test, y_test_pred,
    # col, vectorizer_name, classifier_name,
    pos_label=None, labels=None
):

    if pos_label is None:
        pos_label = 1
    if labels is None:
        labels = [1, 0]

    # Using y_pred
    explained_variance = metrics.explained_variance_score(y_test, y_test_pred)
    accuracy = metrics.accuracy_score(y_test, y_test_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_test_pred)
    precision = metrics.precision_score(y_test, y_test_pred, pos_label=1, labels=[1, 0])
    recall = metrics.recall_score(y_test, y_test_pred, pos_label=1, labels=[1, 0])
    f1 = metrics.f1_score(y_test, y_test_pred)
    mcc = metrics.matthews_corrcoef(y_test, y_test_pred)
    fm = metrics.fowlkes_mallows_score(y_test, y_test_pred)
    kappa = metrics.cohen_kappa_score(y_test, y_test_pred)
    gmean_iba = imblearn.metrics.make_index_balanced_accuracy(alpha=0.1, squared=True)(geometric_mean_score)
    gmean = gmean_iba(y_test, y_test_pred)
    report = metrics.classification_report(y_test, y_test_pred)
    cm = metrics.confusion_matrix(y_test, y_test_pred)
    cm_normalized = metrics.confusion_matrix(y_test, y_test_pred, normalize='true')

    return (
        explained_variance, accuracy, balanced_accuracy, precision,
        recall, f1, mcc, fm, kappa, gmean, report, cm, cm_normalized
    )


In [ ]:
def plot_metrics_with_y_pred(
    y_test, y_test_pred,
    col, vectorizer_name, classifier_name,
    pos_label=None
):
    if pos_label is None:
        pos_label = 1
    # Using y_pred_prob
    # Displays
    close_plots()
    cm_curve = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_test_pred
    )
    cm_normalized_curve = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_test_pred, normalize='true'
    )
    roc_curve = metrics.RocCurveDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    pr_curve = metrics.PrecisionRecallDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    calibration_curve = CalibrationDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    close_plots()

    # Plots
    plots_dict = {
        'Confusion Matrix': cm_curve,
        'Normalized Confusion Matrix': cm_normalized_curve,
        'ROC Curve': roc_curve,
        'Precision-Recall Curve': pr_curve,
        'Calibration Curve': calibration_curve,
    }

    print('=' * 20)
    close_plots()
    print('Plotting metrics with y_pred_prob:')

    for plot_name, plot_ in plots_dict.items():
        print(f'Plotting {plot_name}:')
        fig, ax = plt.subplots()
        ax.set_title(
            f'{str(col)} - {plot_name} {vectorizer_name} + {classifier_name}')
        if plot_name == 'ROC Curve':
            ax.plot([0, 1], [0, 1], 'r--', lw=1)
        plot_.plot(ax=ax)
        show_and_close_plots()
        print('=' * 20)

        # Save Plots
        print(f'Saving {plot_name}...')
        for image_save_format in ['eps', 'png']:
            plt.savefig(
                f'{plot_save_path}{method} {str(col)} - {plot_name} {vectorizer_name} + {classifier_name}.{image_save_format}',
                format=image_save_format, dpi=3000, bbox_inches='tight'
            )
        print(f'Saved {plot_name}!')
        print('=' * 20)

    # # Visualisation with plot_metric
    # bc = plot_metric.functions.BinaryClassification(
    #     y_test, y_test_pred, labels=[0, 1])

    # # Figures
    # fig = plt.figure(figsize=(15, 10))
    # plt.subplot2grid((2, 6), (1, 1), colspan=2)
    # bc.plot_confusion_matrix(colorbar=True)
    # plt.subplot2grid((2, 6), (1, 3), colspan=2)
    # bc.plot_confusion_matrix(normalize=True, colorbar=True)
    # plt.subplot2grid(shape=(2, 6), loc=(0, 0), colspan=2)
    # bc.plot_roc_curve()
    # plt.subplot2grid((2, 6), (0, 2), colspan=2)
    # bc.plot_precision_recall_curve()
    # plt.subplot2grid((2, 6), (0, 4), colspan=2)
    # bc.plot_class_distribution()
    # show_and_close_plots()
    # bc.print_report()
    # for image_save_format in ['eps', 'png']:
    #     plt.savefig(
    #         f'{plot_save_path}{method} plot_metric Curves {str(col)} - {vectorizer_name} + {classifier_name}.{image_save_format}',
    #         format=image_save_format,
    #         dpi=3000, bbox_inches='tight'
    #     )

    # Heatmap
    print('Plotting Heatmap:')
    close_plots()
    classifications_dict = defaultdict(int)
    for _y_test, _y_test_pred in zip(y_test, y_test_pred):
        if _y_test != _y_test_pred:
            classifications_dict[(_y_test, _y_test_pred)] += 1

    dicts_to_plot = [
        {
            f'True {str(col)} value': _y_test,
            f'Predicted {str(col)} value': _y_test_pred,
            'Number of Classifications': _count,
        }
        for (_y_test, _y_test_pred), _count in classifications_dict.items()
    ]
    df_to_plot = pd.DataFrame(dicts_to_plot)
    df_wide = df_to_plot.pivot_table(
        index=f'True {str(col)} value', 
        columns=f'Predicted {str(col)} value', 
        values='Number of Classifications'
    )

    plt.figure(figsize=(9,7))
    sns.set(style='ticks', font_scale=1.2)
    sns.heatmap(df_wide, linewidths=1, cmap='Purples')    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    show_and_close_plots()

    return (
        cm_curve, cm_normalized_curve, roc_curve, pr_curve, calibration_curve
    )


In [ ]:
def compute_metrics_with_y_pred_prob(
    y_test, y_test_pred_prob,
    # col, vectorizer_name, classifier_name,
    pos_label=None
):

    if pos_label is None:
        pos_label = 1

    # Using y_pred_prob
    average_precision = metrics.average_precision_score(y_test, y_test_pred_prob)
    roc_auc = metrics.roc_auc_score(y_test, y_test_pred_prob)
    auc = metrics.auc(fpr, tpr)
    fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_pred_prob, pos_label=1)
    loss = metrics.log_loss(y_test, y_test_pred_prob)
    precision_pr, recall_pr, threshold_pr = metrics.precision_recall_curve(y_test, y_test_pred_prob, pos_label=1)

    return (
        average_precision, roc_auc, auc,
        fpr, tpr, threshold,loss,
        precision_pr, recall_pr, threshold_pr
    )


In [ ]:
def compute_metrics(
    predicted_results, 
    with_y_pred=None,
    with_y_pred_prob=None
):

    if with_y_pred is None:
        with_y_pred = True
    if with_y_pred_prob is None:
        with_y_pred_prob = False
    
    # Get y_test_pred
    y_test = predicted_results.label_ids
    y_test_pred = predicted_results.predictions.argmax(-1)
    y_test_pred = y_test_pred.flatten().tolist()
    y_test_pred = [bert_label2id[l] for l in y_test_pred]

    # # Get y_test_pred_prob
    # try:
    #     y_test_pred_prob = torch.nn.functional.softmax(torch.tensor(predicted_results.predictions, device=device), dim=-1)
    #     print('Using torch.nn.functional.softmax')
    # except Exception:
    #     y_test_pred_prob = scipy.special.softmax(predicted_results.predictions, axis=1)
    #     print('Using scipy.special.softmax')
    # except Exception:
    #     y_test_pred_prob = predicted_results.predictions[:, 1]
    #     print('Using predicted_results.predictions[:, 1]')
    # finally:
    #     y_test_pred_prob = y_test_pred_prob.flatten().tolist()

    # Get metrics
    print('-'*20)
    # Using y_test_pred
    if with_y_pred:
        print('Computing metrics using y_test_pred.')
        (
            explained_variance, accuracy, balanced_accuracy, precision,
            recall, f1, mcc, fm, kappa, gmean, report, cm, cm_normalized
        ) = compute_metrics_with_y_pred(
            y_test, y_test_pred,
        )
    # Using y_test_pred_prob
    if with_y_pred_prob:
        print('Computing metrics using y_test_pred_prob.')
        (
            average_precision, roc_auc, auc,
            fpr, tpr, threshold, loss,
            precision_pr, recall_pr, threshold_pr
        ) = compute_metrics_with_y_pred_prob(
            y_test, y_test_pred_prob,
        )

    # Place metrics into dict
    print('-'*20)
    print('Appending metrics to dict.')
    metrics_dict = {
        # 'Mean Cross Validation Train Score': float(cv_train_scores),
        # 'Mean Cross Validation Test Score': float(cv_test_scores),
        # f'Mean Cross Validation Train - {scoring.title()}': float(cv_train_recall),
        # f'Mean Cross Validation Test - {scoring.title()}': float(cv_test_recall),
        # f'Mean Explained Train Variance - {scoring.title()}': float(cv_train_explained_variance_recall),
        # f'Mean Explained Test Variance - {scoring.title()}': float(cv_test_explained_variance_recall),
        # 'Explained Variance': float(explained_variance),
        'Accuracy': float(accuracy),
        'Balanced Accuracy': float(balanced_accuracy),
        'Precision': float(precision),
        # 'Average Precision': float(average_precision),
        'Recall': float(recall),
        'F1-score': float(f1),
        'Matthews Correlation Coefficient': float(mcc),
        'Fowlkes–Mallows Index': float(fm),
        # 'ROC': float(roc_auc),
        # 'AUC': float(auc),
        # f'{scoring.title()} Best Threshold': threshold,
        # f'{scoring.title()} Best Score': float(best_score),
        # 'Log Loss/Cross Entropy': float(loss),
        'Cohen’s Kappa': float(kappa),
        'Geometric Mean': float(gmean),
        'Classification Report': report,
        'Confusion Matrix': cm,
        'Normalized Confusion Matrix': cm_normalized
    }

    return metrics_dict


In [ ]:
def plot_metrics(
    estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
    col, vectorizer_name, classifier_name, 
    with_y_pred=None, with_y_pred_prob=None
):
    if with_y_pred is None:
        with_y_pred = True
    if with_y_pred_prob is None:
        with_y_pred_prob = False

    # Plotting
    # Using y_test_pred
    if with_y_pred:
        (
            cm_curve, cm_normalized_curve, roc_curve, pr_curve, calibration_curve
        ) = plot_metrics_with_y_pred(
            y_test, y_test_pred,
            col, vectorizer_name, classifier_name, 
        )


In [ ]:
def evaluation(metrics_dict, df_metrics, col, vectorizer_name, classifier_name):

    # Print metrics
    print('=' * 20)
    print('~' * 20)
    print(' Metrics:')
    print('~' * 20)
    print(f'Classification Report:\n {metrics_dict["Classification Report"]}')
    print('-' * 20)
    for test_metric_name, metric_value in metrics_dict.items():
        if test_metric_name not in ['test_runtime', 'test_samples_per_second', 'test_steps_per_second']:
            metric_name = test_metric_name.split("test_")[1].replace('_', ' ').title()
            if isinstance(metric_name, float):
                print(f'{metric_name}: {round(float(metric_value), 2)}')
            else:
                print(f'{metric_name}: {metric_value}')
            print('-' * 20)

            # Fill Table DF
            if isinstance(metric_value, float):
                df_metrics.loc[
                    (classifier_name), (col, vectorizer_name, metric_name)
                ] = float(metric_value)
            else:
                df_metrics.loc[
                    (classifier_name), (col, vectorizer_name, metric_name)
                ] = str(metric_value)

    print('=' * 20)

    # Plot Metrics
    plot_metrics(
        estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
        col, vectorizer_name, classifier_name, 
    )

    return df_metrics


In [ ]:
# Save Model
def saving_model_and_table(df_metrics, estimator, col, vectorizer_name, classifier_name):

    # Save metrics df
    print(f'Saving Model and Table for {vectorizer_name} + {classifier_name}.')
    df_metrics.to_csv(f'{table_save_path}Classifiers Table.csv')
    df_metrics.to_pickle(f'{table_save_path}Classifiers Table.pkl')
    df_metrics.to_excel(f'{table_save_path}Classifiers Table.xlsx')
    df_metrics.to_latex(f'{table_save_path}Classifiers Table.tex')
    df_metrics.to_markdown(f'{table_save_path}Classifiers Table.md')

    # Save estimator
    estimator.save_model(f'{models_save_path}{method} Estimator {str(col)} - {vectorizer_name} + {classifier_name})')


In [ ]:
# Save Model
def get_fitted_estimators():
    
    estimators_list = []

    for model_path in glob.glob(f'{models_save_path}*.pkl'):
        with open(model_path, 'rb') as f:
            estimators_list.append(joblib.load(f))

    return estimators_list


In [ ]:
def comparison_plots(
    estimators_list, X_test, y_test, col,
    curves_dict=None, cmap=plt.cm.Blues
):

    curves_dict = {
        'ROC Curve': metrics.RocCurveDisplay,
        'Precision Recall Curve': metrics.PrecisionRecallDisplay,
        'Calibration Curve': metrics.CalibrationDisplay,
    }

    assert len(estimators_list) != 0

    for curve_name, curve_package in curves_dict.items():
        print('-' * 20)
        print(f'{str(curve_name)}: {str(col)}')
        fig, ax = plt.subplots()
        ax.set_title(f'{str(curve_name)}: {str(col)}')
        for estimator in estimators_list:
            curve = curve_package.from_estimator(
                estimator, X_test, y_test, pos_label=1, ax=ax,
                name=f'{estimator.steps[0][0]} + {estimator.steps[1][0]} + {estimator.steps[-1][0]}'
            )
        show_and_close_plots()

        # Save Plots
        print('Saving plots.')
        for image_save_format in ['eps', 'png']:
            curve.figure_.savefig(
                f'{plot_save_path}{method} {str(col)} - All {str(curve_name)}s.{image_save_format}',
                format=image_save_format,
                dpi=3000, bbox_inches='tight'
            )


# Training

### READ DATA

In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl').reset_index(drop=True)
# HACK REMOVE THIS!!!!!!
df_manual = df_manual.groupby(analysis_columns).sample(n=200).reset_index(drop = True)


In [ ]:
%%time
print('#'*40)
print('Starting!')
print('#'*40)

analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'

with joblib.parallel_backend(backend='multiprocessing', n_jobs=n_jobs):
    # Load Table DF
    df_metrics = pd.read_pickle(f'{table_save_path}Classifiers Table.pkl')

    for col in tqdm.tqdm(analysis_columns):

        print('-'*20)
        print(f'{"="*30} TRAINING {col.upper()} {"="*30}')
        print('-'*20)

        assert len(df_manual[df_manual[str(col)].map(df_manual[str(col)].value_counts() > 1)]) != 0

        # Split
        (
            train, X_train, X_train_bert_encodings, y_train, y_train_bert_encoded, bert_train_dataset,
            test, X_test, X_test_bert_encodings, y_test, y_test_bert_encoded, bert_test_dataset,
            val, X_val, X_val_bert_encodings, y_val, y_val_bert_encoded, bert_val_dataset,
            bert_label2id, bert_id2label, class_weights, class_weights_ratio, class_weights_dict
        ) = encode_data(
            df=df_manual, col=col, analysis_columns=analysis_columns, text_col=text_col
        )

        # Initialize Model
        print(f'Initializing BERT Model.')
        tokenizer_name = bert_tokenizer.__class__.__name__
        classifier_name = bert_model.__class__.__name__

        # Load pre-trained BERT model
        bert_model = BertForSequenceClassification.from_pretrained(
            bert_model_name, num_labels=len(bert_id2label)
        ).to(device)
        # bert_model.eval()

        # Name tokenizer and classifier name
        tokenizer_name = bert_tokenizer.__class__.__name__
        classifier_name = bert_model.__class__.__name__

        print('='*30)
        print(f'Initializing BERT Trainer using {tokenizer_name} + {classifier_name} for {col}')

        # Set BERT fine-tuning parameters
        bert_training_args = TrainingArguments(
            output_dir=f'{models_save_path}{method} Results',
            logging_dir=f'{models_save_path}{method} Logs',
            num_train_epochs=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=20,
            learning_rate=5e-5,
            warmup_steps=100,
            weight_decay=0.01,
            logging_steps=100,
            evaluation_strategy='steps',
            optim='adamw_torch',
            use_mps_device=True if device.type == 'mps' else False
        )

        # Pass data to trainer 
        # FIXME: computer metrics not working
        print('-'*20)
        print('Passing arguments to estimator.')
        estimator = Trainer(
            model=bert_model,
            tokenizer=bert_tokenizer,
            args=bert_training_args,
            train_dataset=bert_train_dataset,
            eval_dataset=bert_test_dataset,
            compute_metrics=compute_metrics,
            accelerator=device_name
        )
        if estimator.place_model_on_device:
            estimator.model.to(device)

        # Train trainer
        print('-'*20)
        print(f'Starting training for {col}')
        estimator.train()
        estimator.save_model(f'{models_save_path}{method} Estimator {str(col)} - {tokenizer_name} + {classifier_name})')
        print('Done training!')
        print()

        # Get predictions
        print('-'*20)
        print(f'Evaluating estimator for {col}')
        estimator.evaluate()
        # metrics_dict
        predicted_results = estimator.predict(bert_test_dataset)
        print(f'Predictions shape for {col}: {predicted_results.predictions.shape}')
        print()
        
        # Get y_test_pred
        print('-'*20)
        print(f'Getting y_test_pred for {col}')
        y_test = predicted_results.label_ids
        y_test_pred = predicted_results.predictions.argmax(-1)
        y_test_pred = y_test_pred.flatten().tolist()
        y_test_pred = [bert_label2id[l] for l in y_test_pred]
        print(f'Length of y_test_pred: {len(y_test_pred)}')
        print()

        # Get y_test_pred_proba
        print('-'*20)
        print(f'Getting y_test_pred_prob for {col}')
        try:
            y_test_pred_prob = torch.nn.functional.softmax(predicted_results.predictions.clone().detach().to(device), dim=-1)
            print('Using torch.nn.functional.softmax')
        except Exception:
            y_test_pred_prob = scipy.special.softmax(predicted_results.predictions.clone().detach().to(device), axis=1)
            print('Using scipy.special.softmax')
        except Exception:
            y_test_pred_prob = predicted_results.predictions.clone().detach().to(device)[:, 1]
            print('Using predicted_results.predictions[:, 1]')
        finally:
            y_test_pred_prob = y_test_pred_prob.flatten().tolist()
        print(f'Length of y_test_pred_prob: {len(y_test_pred_prob)}')
        print()

        # HACK
        # # Get y_test_pred_proba
        # print('-'*20)
        # print(f'Getting y_test_pred_prob for {col}')
        # try:
        #     y_test_pred_prob = torch.nn.functional.softmax(torch.tensor(predicted_results.predictions, device=device), dim=-1)
        #     print('Using torch.nn.functional.softmax')
        # except Exception:
        #     y_test_pred_prob = scipy.special.softmax(predicted_results.predictions, axis=1)
        #     print('Using scipy.special.softmax')
        # except Exception:
        #     y_test_pred_prob = predicted_results.predictions[:, 1]
        #     print('Using predicted_results.predictions[:, 1]')
        # finally:
        #     y_test_pred_prob = y_test_pred_prob.flatten().tolist()
        # print(f'Length of y_test_pred_prob: {len(y_test_pred_prob)}')
        # print()
        
        # Examine predictions
        print('-'*20)
        print(f'Examining predictions for {col}')
        print('Correctly Classified Reviews:')
        for y_test, y_test_pred, _X_test in random.sample(list(zip(y_test, y_test_pred, X_test)), 20):
            if y_test == y_test_pred:
                print(f'LABEL: {y_test}')
                print(f'REVIEW TEXT: {_X_test[:100]}...')
                print('-'*20)
                print()
        
        print('Incorrectly Classified Reviews:')
        for y_test, y_test_pred, _X_test in random.sample(list(zip(y_test, y_test_pred, X_test)), 20):
            if y_test != y_test_pred:
                print(f'TRUE LABEL: {y_test}')
                print(f'PREDICTED LABEL: {y_test_pred}')
                print(f'REVIEW TEXT: {_X_test[:100]}...')
                print()

        # Evluate estimator
        print('-'*20)
        print(f'Probs evaluation and ploting metrics for {col}')
        df_metrics = evaluation(predicted_results, df_metrics, col, tokenizer_name, classifier_name)
        print()

        # Save BERT Model
        print('-'*20)
        print(f'Saving estimator and metrics table for {col}')
        saving_model_and_table(df_metrics, estimator, col, tokenizer_name, classifier_name)
        print()

        # Compare Estimators
        print('='*20)
        print(f'Comparing estimators for {col}')
        comparison_plots(get_fitted_estimators().append(estimator), X_test, y_test, col)
        print('='*20)
        print()

print('#'*40)
print('DONE!')
print('#'*40)



In [ ]:
# Get predictions
print('-'*20)
print(f'Evaluating estimator for {col}')
estimator.evaluate()
# metrics_dict
predicted_results = estimator.predict(bert_test_dataset)
print(f'Predictions shape for {col}: {predicted_results.predictions.shape}')

# Get y_test_pred
print('-'*20)
print(f'Getting y_test_pred for {col}')
y_test = predicted_results.label_ids
y_test_pred = predicted_results.predictions.argmax(-1)
y_test_pred = y_test_pred.flatten().tolist()
y_test_pred = [bert_label2id[l] for l in y_test_pred]
print(f'Length of y_test_pred: {len(y_test_pred)}')



In [ ]:
# Get y_test_pred_proba
print('-'*20)
print(f'Getting y_test_pred_prob for {col}')
try:
    y_test_pred_prob = torch.nn.functional.softmax(torch.tensor(predicted_results.predictions, device=device), dim=-1)
    print('Using torch.nn.functional.softmax')
except Exception:
    y_test_pred_prob = scipy.special.softmax(predicted_results.predictions, axis=1)
    print('Using scipy.special.softmax')
except Exception:
    y_test_pred_prob = predicted_results.predictions[:, 1]
    print('Using predicted_results.predictions[:, 1]')
finally:
    y_test_pred_prob = y_test_pred_prob.flatten().tolist()
print(f'Length of y_test_pred_prob: {len(y_test_pred_prob)}')



In [ ]:
predicted_results.metrics

In [ ]:
# Evluate estimator
print('-'*20)
print(f'Probs evaluation and ploting metrics for {col}')
df_metrics = evaluation(predicted_results, df_metrics, col, classifier_name, tokenizer_name)


In [ ]:
estimator.is_model_parallel


In [ ]:
estimator.args.n_gpu

In [ ]:
df_manual['Bert Preditcions'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: estimator.predict(sent)
    for sent in sentence
    if sent and isinstance(sent, (str, list)) and isinstance(sentence, list)
)